Authors:
    <br>Alejandro Alvarez (axa)
    <br>Brenda Palma (bpalmagu)

# <center>ML-Jokes: Content-Based Filtering</center>

In [1]:
# Path to ml-jokes folder
import os
os.chdir('..')
print(os.getcwd())

/home/alejandroxag/my_files/heinz-95729-project/api/ml-jokes


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from mljokes.data import read_jokes, read_ratings

### Load and prepare data

In [3]:
# Load jokes
jokes_df = read_jokes()
display(jokes_df)

,text,clean_text,len
0,One Sunday morning William burst into the livi...,one sunday morning william burst living room s...,218
1,Q: How do you keep a computer programmer in th...,q keep computer programmer shower day long giv...,27
2,"A horse walks into a bar. Bartender says: ""So,...",horse walk bar bartender say long face,13
3,Q: How many programmers does it take to change...,q many programmer take change lightbulb none h...,17
4,"Just a thought .. Before criticizing someone, ...",thought criticizing someone walk mile shoe cri...,30
...,...,...,...
95,"This guys wife asks, ""Honey if I died would yo...",guy wife asks honey died would remarry reply w...,153
96,Why are there so many Jones's in the phone boo...,many jones phone book phone,15
97,Q. What's the difference between a man and a t...,q difference man toilet toilet follow around use,21
98,A dog walks into Western Union and asks the cl...,dog walk western union asks clerk send telegra...,59


In [4]:
# Load ratings
ratings_df = read_ratings()
display(ratings_df)

./data/ratings/jester-data-3.xls
./data/ratings/jester-data-2.xls
./data/ratings/jester-data-1.xls


,count_rated,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,26,99.00,99.00,99.00,99.00,-1.65,99.00,-0.78,6.89,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
1,33,99.00,99.00,99.00,99.00,-9.27,99.00,-9.17,-8.59,99.00,...,99.00,99.00,-2.77,99.00,99.00,99.00,99.00,99.00,99.00,99.00
2,16,99.00,99.00,99.00,99.00,-6.12,99.00,-7.48,-7.77,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
3,24,99.00,0.05,99.00,99.00,-2.82,99.00,-4.85,-0.87,99.00,...,99.00,99.00,99.00,99.00,1.84,99.00,99.00,99.00,-4.08,99.00
4,22,99.00,99.00,99.00,99.00,-4.95,99.00,6.21,2.72,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73416,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
73417,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
73418,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
73419,37,99.00,99.00,99.00,99.00,-9.71,99.00,4.56,-8.30,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [5]:
# Encode text
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(jokes_df.clean_text)
vectorizer.get_feature_names_out()

array(['_num_', 'abortion', 'accompanied', ..., 'younger', 'zipper', 'zo'],
      dtype=object)

In [6]:
# Find latent topics
lda = LatentDirichletAllocation(n_components=5,random_state=0)
X_topics = lda.fit_transform(X)

### Regression model to predict ratings